In [1]:
# importing libraries

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets
import requests


In [ ]:
#Import or Load the Data
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
confirmed_df.set_index(confirmed_df['Province_State'],inplace=True)
confirmed_df.rename(columns={'Province_State': 'State'},inplace=True)
death_df.set_index(death_df['Province_State'],inplace=True)
death_df.rename(columns={'Province_State': 'State'},inplace=True)
confirmed_df_states=confirmed_df.drop(columns=['UID', 'code3', 'FIPS', 'Lat', 'Long_']).groupby(['State']).sum().copy()
death_df_states=death_df.drop(columns=['UID', 'code3', 'FIPS', 'Lat', 'Long_']).groupby(['State']).sum().copy()
confirmed_df_states=pd.merge(confirmed_df[['Lat', 'Long_','State']].groupby(['State']).mean(),confirmed_df_states, left_index=True, right_index=True)
death_df_states=pd.merge(death_df[['Lat', 'Long_','State']].groupby(['State']).mean(),death_df_states, left_index=True, right_index=True)
combined_df=pd.DataFrame()
combined_df['Confirmed']=confirmed_df_states[confirmed_df_states.columns[-1]]
combined_df['Deaths']=death_df_states[death_df_states.columns[-1]]


In [ ]:
#Statewise Data
sorted_combined_df=combined_df.loc[:,["Confirmed","Deaths"]].sort_values("Confirmed",ascending=False)
sorted_combined_df['State']=sorted_combined_df.index
combined_df.loc[:,["Confirmed","Deaths"]].sort_values("Confirmed",ascending=False).style.background_gradient(cmap='Blues',subset=["Confirmed"]).background_gradient(cmap='Reds',subset=["Deaths"])

In [ ]:
world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['Lat'], confirmed_df.iloc[i]['Long_']],
        fill=True,
        radius=int((np.log2(confirmed_df.iloc[i,-1]+1))*6000),
        color='orange',
        fill_color='#ff8533',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.index[i] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map

In [ ]:
state_geo = requests.get('https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json').json()

In [ ]:
data_temp = confirmed_df.groupby(["State"]).sum().reset_index().drop(["Lat","Long_"],axis=1)
data_temp["Confirmed_log"] = np.log10(data_temp.iloc[:,-1]+1)
bins = list(data_temp['Confirmed_log'].quantile([0, 0.25, 0.5, 0.75,0.95 ,1]))
m = folium.Map(location=[37, -102], zoom_start=4,max_zoom=6,min_zoom=3)
 

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data = data_temp,
    columns=['State', 'Confirmed_log'],
    key_on='feature.properties.name',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.2,
    bins = bins,
    reset=True,
    legend_name='Confirmed cases log (10^x)'
).add_to(m)
folium.LayerControl().add_to(m)

legend_html = "<div style='padding:10px;background-color:rgba(255,255,255,0.5);position:fixed;bottom:20px;left:20px;z-index:1000'>"
legend_html += "<div style='width:100%;text-align:center;'><h4>Index for Legend</h4></div><hr style='border-top-color: rgba(25,25,25,0.5);'>"
legend_html += "<ul style='margin:0;padding:0;color: #555;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"
for i in bins:
    legend_html += "<li style='margin:0;padding:0;line-height: 0;'>Value "+str(np.round(i,2))+" : "+str(int(10**i)-1)+"</li><br>"
legend_html += "</ul></div>"
m.get_root().html.add_child(folium.Element(legend_html))
m

In [ ]:
data_temp = death_df.groupby(["State"]).sum().reset_index().drop(["Lat","Long_"],axis=1)
data_temp["Death_log"] = np.log10(data_temp.iloc[:,-1]+1)
bins = list(data_temp['Death_log'].quantile([0, 0.25, 0.5, 0.75,0.95 ,1]))
m = folium.Map(location=[37, -102], zoom_start=4,max_zoom=6,min_zoom=3)
 

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data = data_temp,
    columns=['State', 'Death_log'],
    key_on='feature.properties.name',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.2,
    bins = bins,
    reset=True,
    legend_name='Death cases log (10^x)'
).add_to(m)
folium.LayerControl().add_to(m)

legend_html = "<div style='padding:10px;background-color:rgba(255,255,255,0.5);position:fixed;bottom:20px;left:20px;z-index:1000'>"
legend_html += "<div style='width:100%;text-align:center;'><h4>Index for Legend</h4></div><hr style='border-top-color: rgba(25,25,25,0.5);'>"
legend_html += "<ul style='margin:0;padding:0;color: #555;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"
for i in bins:
    legend_html += "<li style='margin:0;padding:0;line-height: 0;'>Value "+str(np.round(i,2))+" : "+str(int(10**i)-1)+"</li><br>"
legend_html += "</ul></div>"
m.get_root().html.add_child(folium.Element(legend_html))
m

In [ ]:
#Dashboard for Washington
df_wa = pd.read_csv("~/Desktop/wa_time_series_covid.csv")
df_wa = df_wa.drop(df_wa.index[-1])

In [ ]:
#Total number of confirmed death and cases
total_cases = int(df_wa['Cases'].sum())
deaths_total = int(df_wa['Deaths'].sum())
hospitalization_total = int(df_wa['Hospitalizations'].sum())

In [ ]:
#display Stats
display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Cases: "  + str(total_cases) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Hospitalizations: " + str(hospitalization_total) + "</span>"+
             "</div>"))

In [ ]:
#Enter the Number of counties you want the data for 
# sorting the values by confirmed descednding order
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: yellow'
    b = 'background-color: blue'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 1] = y
    df1.iloc[:, 3] = r
    df1.iloc[:, 2] = b
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return df_wa.sort_values('Cases', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

In [ ]:
sorted_county_df =df_wa.sort_values('Cases', ascending= False)

In [ ]:

import geopandas as gpd

#Get the data
data = pd.read_csv("~/Desktop/wa_time_series_covid.csv")

# Read the county shapefile for Washington state
counties = gpd.read_file('~/Downloads/Washington_Counties_with_Natural_Shoreline___washsh_area.geojson')[['COUNTY','geometry']]

#Rename the columns
counties.columns = ['county','geometry']

In [ ]:
data['County'] = data['County'].apply(str.upper)
data.columns = [x.lower() for x in data.columns]

In [ ]:
#Preprocess the Data
# Print the map
#counties.plot(figsize=(20, 20), edgecolor='white', linewidth=1, color='lightblue')

In [ ]:
#merge the shapefile and the data
geo_county = counties.merge(data,on='county').dropna(axis=0).sort_values(by='cases',ascending=False)


In [ ]:
geo_county.plot(column='cases',scheme="quantiles",figsize=(10,10),cmap='Spectral',alpha = 0.5,edgecolor='k',legend=True)
plt.title('COVID-19 Confirmed Cases by County',fontsize=25)
#leg = ax.get_legend()
#leg.set_bbox_to_anchor((1.3,1))
plt.show()

In [ ]:
@interact(hue = ['cases','hospitalizations','deaths'])
def plot(hue):
    _ = sns.pairplot(geo_county, hue = hue)

In [ ]:
## Slide to hit the worst hit counties
# # plotting the 20 worst hit counties

def bubble_chart(n):
    fig = px.scatter(sorted_county_df.head(n), x="County", y="Cases", size="Cases", color="County",
               hover_name="County", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit counties",
    xaxis_title="Counties",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)